https://github.com/ggerganov/llama.cpp/discussions/2948
https://hackernoon.com/quantizing-large-language-models-with-llamacpp-a-clean-guide-for-2024


In [1]:
!pip install huggingface_hub

In [2]:
from huggingface_hub import snapshot_download

author = "stanford-crfm"
modelname = "BioMedLM"
model_id = author + "/" + modelname

snapshot_download(repo_id=model_id, local_dir="hf-model",
                  local_dir_use_symlinks=False, revision="main")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.78k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/602k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/267 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/276k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

'/content/hf-model'

In [3]:
!ls -lash hf-model

total 10G
4.0K drwxr-xr-x 2 root root 4.0K Apr  5 02:20 .
4.0K drwxr-xr-x 1 root root 4.0K Apr  5 02:20 ..
4.0K -rw-r--r-- 1 root root  876 Apr  5 02:19 config.json
4.0K -rw-r--r-- 1 root root 1.5K Apr  5 02:19 .gitattributes
272K -rw-r--r-- 1 root root 270K Apr  5 02:19 merges.txt
 10G -rw-r--r-- 1 root root  10G Apr  5 02:20 pytorch_model.bin
 12K -rw-r--r-- 1 root root 9.6K Apr  5 02:19 README.md
4.0K -rw-r--r-- 1 root root  267 Apr  5 02:19 tokenizer_config.json
1.2M -rw-r--r-- 1 root root 1.2M Apr  5 02:19 tokenizer.json
588K -rw-r--r-- 1 root root 588K Apr  5 02:19 vocab.json


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements.txt

In [5]:
!mkdir ./quantized_model/
!python llama.cpp/convert-hf-to-gguf.py ./hf-model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

Loading model: hf-model
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
gguf: Adding 28639 merge(s).
gguf: Setting special token type bos to 28895
gguf: Setting special token type eos to 28895
gguf: Setting special token type unk to 28895
Exporting model to 'quantized_model/FP16.gguf'
gguf: loading model part 'pytorch_model.bin'
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(insta

In [6]:
import os

methods = ["q5_k_m"]
quantized_path = "./quantized_model/"

for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [9]:
!zip -r /content/quantized.zip /content/quantized_model

  adding: content/quantized_model/ (stored 0%)
  adding: content/quantized_model/Q5_K_M.gguf (deflated 0%)
  adding: content/quantized_model/FP16.gguf (deflated 8%)


In [10]:
from google.colab import files

files.download("/content/quantized.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>